<a href="https://colab.research.google.com/github/carlibeisel/pod_pou_lulcc/blob/main/03_subset_LULCC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Modified by Carli Beisel

Adapted from code written by Bridget Bittmann (2023, Github: bridgetmarie24)

Date originally created: March 14, 2022

Date modified: April 16, 2024

Purpose: This script uses LCMAP data to calculate annual percent of each land class from 1987 to 2020 for each POU

# Install Libraries

You must install the correct version of pylandstats (2.1.3)

In [1]:
## IMPORT PACKAGES ##

import numpy as np #basic computation
!pip install geopandas
import geopandas as gpd #geopandas for .shp
import matplotlib.pyplot as plt #to create plots
import pandas as pd #to create dataframes and export .csv
!pip install rasterio
import rasterio as rso #import GeoTiff files
from rasterio.mask import mask #to crop data to a boundary
from rasterio.plot import show #to plot the image
from rasterio.crs import CRS
from shapely.ops import unary_union #creates boundary of shapefile
import json #imports metadata
!pip install rioxarray #to clip rasters to a .shp file
import rioxarray as rxr
from rasterio.warp import calculate_default_transform, reproject, Resampling
!pip install pylandstats==2.1.3 #very important
import pylandstats #to perform landscape metrics
from pylandstats import landscape
from pylandstats import SpatioTemporalAnalysis #to calculate landscape metrics through time
import glob
import os
import matplotlib.lines as lines
import matplotlib.patches as patch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 2.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached transonic-0.6.4-py3-none-any.whl (74 kB)
  Using cached autopep8-2.1.0-py2.py3-none-any.whl (44 kB)
  Using cached beniget-0.4.1-py3-none-any.whl (9.4 kB)
  Using cached pycodestyle-2.11.1-py2.py3-none-any.whl (31 kB)
  Created wheel for pylandstats: filename=pylandstats-2.1.3-cp310-cp310-linux_x86_64.whl size=65649 sha256=f36d2a1467ffacb75247104754ef0f0bcbb4da0cadaabd3ba126e770b8ee0c61
  Stored in directory: /root/.cache/pip/wheels/34/f6/d2/09bb553889855a679e8f776a622b9dc3818bdd0182a4e40499
Successfully built pylandstats


In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [3]:
## ---------------------- ##
## SUBSET GEOSPATIAL DATA ##
## ---------------------- ##
import rasterio
with rasterio.Env(GTIFF_SRS_SOURCE='EPSG'): #added to project all in same EPSG coordinate system
    shp_file = gpd.read_file('/content/gdrive/MyDrive/Data/Datasets from Bridget/pod_pou_lulcc/POUs/POUs_EDIT_060622_Merge.shp') #open shapefile
    names = shp_file['WaterRight']
    files = glob.glob('/content/gdrive/MyDrive/Data/Datasets from Bridget/pod_pou_lulcc/lcmap_files/lcmaps/*.tiff') #get all the years of cdl imagery
    data =[]
    for i in range(len(files)):
        data.append(rasterio.open(files[i])) #open cdl image and append to a list
        print(rasterio.open(files[i])) #added to check
    shp = shp_file.to_crs(data[1].crs) #reproject the shp file to same projection
    years = np.arange(1987, 2021) #years of LCMAP data
    collection = []
    for i in range(len(shp)):
        for n in range(len(years)):
            dataset = data[n]
            year_out = dataset.name[93:97] #change these numbers if things aren't working
            print(year_out) #added
            extent = gpd.GeoSeries(shp['geometry'][i]) #get the geometry from shapefile
            coords = [json.loads(extent.to_json())['features'][0]['geometry']] #gets coordinates for rasterio input
            out_img, out_transform = mask(dataset=dataset, shapes=coords, crop=True, nodata=0) #crop the data to the shapefile
            out_meta = dataset.meta.copy()
            out_meta.update({"driver": "GTiff",
                             "height": out_img.shape[1],
                             "width": out_img.shape[2],
                             "transform": out_transform})
            # Merge original file name with init_landcover to denote that it is the initial land cover data for Janus
            in_file = files[n]
            out_filename = os.path.join('/content/gdrive/MyDrive/Data/Datasets from Bridget/pod_pou_lulcc/lcmap_files/lcmap_masked/'+names[i]+'_'+year_out+'.tif') #create a file name to export to
            print(out_filename) #added
            # Save clipped land cover coverage THIS WILL OVERWRITE FILES
            out_tiff = rasterio.open(out_filename, 'w', **out_meta)
            out_tiff.write(np.squeeze(out_img, 0), 1)
            out_tiff.close()
            collection.append(out_img)

<open DatasetReader name='/content/gdrive/MyDrive/Data/Datasets from Bridget/pod_pou_lulcc/lcmap_files/lcmaps/LCMAP_CU_1987_V12_LCPRI.tiff' mode='r'>
<open DatasetReader name='/content/gdrive/MyDrive/Data/Datasets from Bridget/pod_pou_lulcc/lcmap_files/lcmaps/LCMAP_CU_1986_V12_LCPRI.tiff' mode='r'>
<open DatasetReader name='/content/gdrive/MyDrive/Data/Datasets from Bridget/pod_pou_lulcc/lcmap_files/lcmaps/LCMAP_CU_1989_V12_LCPRI.tiff' mode='r'>
<open DatasetReader name='/content/gdrive/MyDrive/Data/Datasets from Bridget/pod_pou_lulcc/lcmap_files/lcmaps/LCMAP_CU_1988_V12_LCPRI.tiff' mode='r'>
<open DatasetReader name='/content/gdrive/MyDrive/Data/Datasets from Bridget/pod_pou_lulcc/lcmap_files/lcmaps/LCMAP_CU_1990_V12_LCPRI.tiff' mode='r'>
<open DatasetReader name='/content/gdrive/MyDrive/Data/Datasets from Bridget/pod_pou_lulcc/lcmap_files/lcmaps/LCMAP_CU_1991_V12_LCPRI.tiff' mode='r'>
<open DatasetReader name='/content/gdrive/MyDrive/Data/Datasets from Bridget/pod_pou_lulcc/lcmap_fil

In [4]:
## ---------------------------------------- ##
## Import multiple rasters into PyLandStats ##
## ---------------------------------------- ##

years = np.arange(1987,2021)
temporal_group = []
for i in range(len(names)):
  files= sorted(glob.glob('/content/gdrive/MyDrive/Data/Datasets from Bridget/pod_pou_lulcc/lcmap_files/lcmap_masked/'+names[i]+'_*.tif'))
  sta = SpatioTemporalAnalysis(files, dates=years) #import all CDL rasters and mask
  temporal_group.append(sta)


In [5]:
# ------------------------------- #
# CALCULATE THE CLASS PROPORTIONS #
# ------------------------------- #

proportions = []

for i in range(len(names)):
  df = SpatioTemporalAnalysis.compute_class_metrics_df(temporal_group[i], metrics=['proportion_of_landscape'])
  df.to_csv('/content/gdrive/MyDrive/Data/Datasets from Bridget/pod_pou_lulcc/lcmap_files/proportions/'+names[i]+'_prop.csv')
  proportions.append(df)

In [6]:
# ------------------------------- #
#  CALCULATE LARGEST PATCH METRICS
# ------------------------------- #
patch = []

for i in range(len(names)):
  df = SpatioTemporalAnalysis.compute_landscape_metrics_df(temporal_group[i], metrics = ['largest_patch_index'])
  df.to_csv('/content/gdrive/MyDrive/Data/Datasets from Bridget/pod_pou_lulcc/lcmap_files/largest_patch_index/'+names[i]+'_patch.csv')
  patch.append(df)

In [7]:
# ------------------------------- #
#     CALCULATE CONTAGION
# ------------------------------- #
# Original code computed contagion + largest_patch_index in the same step. BUt you
# cannot compute contagion with landscape_metrics.
# Make sure you have the correct version of PyLandStats (2.1.3) or it won't run,
# the error code won't look like its a package version issue.
for i in range(len(names)):
    contag = []
    files = sorted(glob.glob('/content/gdrive/MyDrive/Data/Datasets from Bridget/pod_pou_lulcc/lcmap_files/lcmap_masked/'+names[i]+'_*.tif'))
    k = 0
    for file in files:
        sta = pylandstats.Landscape(file, nodata=0)
        DD = sta.contagion()
        output_file = '/content/gdrive/MyDrive/Data/Datasets from Bridget/pod_pou_lulcc/lcmap_files/contagion/' + names[i] + '_contagion.csv'
        contag.append({'dates': years[k], 'contagion': DD})
        k = k+1
        contag_df = pd.DataFrame(contag)
        contag_df.to_csv(output_file, index=False)

/usr/local/lib/python3.10/dist-packages/pylandstats/landscape.py:2397: RuntimeWarning: Contagion can only be computed in landscapes with more than two classes of patches. Returning nan
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pylandstats/landscape.py:2397: RuntimeWarning: Contagion can only be computed in landscapes with more than two classes of patches. Returning nan
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pylandstats/landscape.py:2397: RuntimeWarning: Contagion can only be computed in landscapes with more than two classes of patches. Returning nan
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pylandstats/landscape.py:2397: RuntimeWarning: Contagion can only be computed in landscapes with more than two classes of patches. Returning nan
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pylandstats/landscape.py:2397: RuntimeWarning: Contagion can only be computed in landscapes with more than two classes of patches. Returning nan
  warni

In [42]:
files_proportions = sorted(glob.glob('/content/gdrive/MyDrive/Data/Datasets from Bridget/pod_pou_lulcc/lcmap_files/proportions/*_prop.csv'))
proportions = []
for i in files_proportions:
  data = pd.read_csv(i)
  proportions.append(data)
`print(data)

In [49]:
# ----------------------------------------------------------------- #
# Put class proportions in the same format as configuration metrics #
# ----------------------------------------------------------------- #

# Import csv files into a list of dataframes
files_proportions = sorted(glob.glob('/content/drive/MyDrive/Data/Datasets from Bridget/pod_pou_lulcc/lcmap_files/proportions/*_prop.csv'))
files_contagion = sorted(glob.glob('/content/drive/MyDrive/Data/Datasets from Bridget/pod_pou_lulcc/lcmap_files/contagion/*_contagion.csv'))
files_patch = sorted(glob.glob('/content/drive/MyDrive/Data/Datasets from Bridget/pod_pou_lulcc/lcmap_files/largest_patch_index/*_patch.csv'))
names = list(sorted(shp_file['WaterRight']))

proportions = []
for i in files_proportions:
  data = pd.read_csv(i)
  proportions.append(data)
contagion = []
for i in files_contagion:
  data = pd.read_csv(i)
  contagion.append(data)
patch = []
for i in files_patch:
  data = pd.read_csv(i)
  patch.append(data)

#Create new dataframes in same format as configuration metrics

new_df = []
for i in range(len(proportions)):
  df = pd.DataFrame(years, columns=['dates'])
  prop = proportions[i]
  df['DivName'] = names[i]
  df['class1_urban'] = prop['proportion_of_landscape'][prop['class_val'] == 1]
  df['class2_crops'] = prop['proportion_of_landscape'][prop['class_val'] == 2].values
  df = df.fillna(0)
  new_df.append(df)

In [54]:
## ------------------------------------------ ##
## CALCULATE CHANGE IN URBAN AREA FOR MAPPING ##
## ------------------------------------------ ##

prop = pd.concat(new_df)

change = prop.groupby('DivName', as_index=False).class1_urban.agg(['min','max']).reset_index().fillna(0)
change['urb_change'] = change['max']-change['min']
change.to_csv('/content/drive/MyDrive/Data/Datasets from Bridget/pod_pou_lulcc/lcmap_files/proportions/change.csv')

In [64]:
## -------------------- ##
## MERGE TWO DATAFRAMES ##
## -------------------- ##

merged = []

for i in range(len(new_df)):
  df = new_df[i]
  contag = contagion[i]
  pat = patch[i]
  df_merge = df.merge(contag, on='dates', how='left').merge(pat, on='dates', how='left')
  display (df_merge)
  df_merge.to_csv('/content/drive/MyDrive/Data/Datasets from Bridget/pod_pou_lulcc/lcmap_files/final_metrics/'+ names[i] +'.csv')
  merged.append(df_merge)

,dates,DivName,class1_urban,class2_crops,contagion,largest_patch_index
0,1987,Andrews,0.871656,97.490231,90.405558,96.964232
1,1988,Andrews,0.901713,97.475203,90.384521,96.949203
2,1989,Andrews,0.931770,97.354974,89.992851,96.828975
3,1990,Andrews,0.946799,97.339946,89.950243,96.813946
4,1991,Andrews,0.961827,97.370003,90.085771,96.844004
5,1992,Andrews,0.976856,97.309889,89.963780,96.798918
6,1993,Andrews,0.961827,97.415089,90.288022,96.904118
7,1994,Andrews,0.961827,97.400060,90.169502,96.889089
8,1995,Andrews,0.931770,97.385032,90.160728,96.874061
9,1996,Andrews,0.901713,97.249775,89.807721,96.723775


,dates,DivName,class1_urban,class2_crops,contagion,largest_patch_index
0,1987,Atwell,0.0,52.380952,14.476105,47.619048
1,1988,Atwell,0.0,52.380952,14.476105,47.619048
2,1989,Atwell,0.0,57.142857,13.834583,42.857143
3,1990,Atwell,0.0,57.142857,13.834583,42.857143
4,1991,Atwell,0.0,61.904762,14.557276,38.095238
5,1992,Atwell,0.0,61.904762,14.557276,38.095238
6,1993,Atwell,0.0,61.904762,14.557276,38.095238
7,1994,Atwell,0.0,61.904762,14.557276,38.095238
8,1995,Atwell,0.0,61.904762,14.557276,38.095238
9,1996,Atwell,0.0,57.142857,13.834583,42.857143


,dates,DivName,class1_urban,class2_crops,contagion,largest_patch_index
0,1987,Ballentyne,20.426245,78.879310,70.836878,67.696360
1,1988,Ballentyne,21.312261,78.017241,70.115077,66.906130
2,1989,Ballentyne,22.078544,77.227011,69.346397,44.037356
3,1990,Ballentyne,22.701149,76.604406,69.124780,43.893678
4,1991,Ballentyne,23.706897,75.598659,68.480077,43.127395
5,1992,Ballentyne,24.233716,75.071839,68.175402,42.935824
6,1993,Ballentyne,24.880268,74.425287,67.668358,42.792146
7,1994,Ballentyne,25.886015,73.419540,61.693190,42.001916
8,1995,Ballentyne,28.256705,70.977011,60.100628,39.727011
9,1996,Ballentyne,28.783525,70.378352,65.457647,39.176245


,dates,DivName,class1_urban,class2_crops,contagion,largest_patch_index
0,1987,Barber,8.839779,91.160221,67.445407,90.055249
1,1988,Barber,8.839779,91.160221,67.445407,90.055249
2,1989,Barber,8.839779,91.160221,67.445407,90.055249
3,1990,Barber,8.839779,91.160221,67.445407,90.055249
4,1991,Barber,8.839779,91.160221,67.445407,90.055249
5,1992,Barber,8.839779,91.160221,67.445407,90.055249
6,1993,Barber,8.839779,91.160221,67.445407,90.055249
7,1994,Barber,8.839779,91.160221,67.445407,90.055249
8,1995,Barber,8.839779,91.160221,67.445407,90.055249
9,1996,Barber,8.839779,91.160221,67.445407,90.055249


,dates,DivName,class1_urban,class2_crops,contagion,largest_patch_index
0,1987,Bates,0.549451,99.450549,91.026841,99.450549
1,1988,Bates,0.549451,99.450549,91.026841,99.450549
2,1989,Bates,0.549451,99.450549,91.026841,99.450549
3,1990,Bates,0.549451,99.450549,91.026841,99.450549
4,1991,Bates,0.549451,99.450549,91.026841,99.450549
5,1992,Bates,0.549451,99.450549,91.026841,99.450549
6,1993,Bates,0.549451,99.313187,93.402456,99.313187
7,1994,Bates,0.549451,99.313187,93.402456,99.313187
8,1995,Bates,0.549451,99.313187,93.402456,99.313187
9,1996,Bates,0.549451,99.175824,92.823071,99.175824


...

,dates,DivName,class1_urban,class2_crops,contagion,largest_patch_index
0,1987,Thurman Mill,58.857143,34.358149,61.568703,52.338028
1,1988,Thurman Mill,60.032193,32.523139,57.406255,53.271630
2,1989,Thurman Mill,60.410463,32.032193,61.590007,53.344064
3,1990,Thurman Mill,61.569416,30.623742,61.851448,54.454728
4,1991,Thurman Mill,62.816901,29.126761,62.383156,55.501006
5,1992,Thurman Mill,63.847082,27.879276,59.006327,56.354125
6,1993,Thurman Mill,64.981891,26.406439,60.048470,57.505030
7,1994,Thurman Mill,65.674044,25.344064,60.648060,60.048290
8,1995,Thurman Mill,67.090543,23.822938,61.795314,61.521127
9,1996,Thurman Mill,67.460765,23.307847,62.209447,62.350101


,dates,DivName,class1_urban,class2_crops,contagion,largest_patch_index
0,1987,Upper Centerpoint,0.071200,98.504806,94.249922,98.504806
1,1988,Upper Centerpoint,0.071200,98.504806,94.249922,98.504806
2,1989,Upper Centerpoint,0.071200,98.504806,94.249922,98.504806
3,1990,Upper Centerpoint,0.071200,98.504806,94.249922,98.504806
4,1991,Upper Centerpoint,0.071200,98.504806,94.249922,98.504806
5,1992,Upper Centerpoint,0.106800,98.469206,94.097475,98.469206
6,1993,Upper Centerpoint,0.106800,98.469206,94.097475,98.469206
7,1994,Upper Centerpoint,0.106800,98.469206,94.097475,98.469206
8,1995,Upper Centerpoint,0.106800,98.504806,94.279484,98.504806
9,1996,Upper Centerpoint,0.106800,98.504806,94.279484,98.504806


,dates,DivName,class1_urban,class2_crops,contagion,largest_patch_index
0,1987,Vale Pump,0.000000,99.208443,90.880036,99.208443
1,1988,Vale Pump,0.000000,99.472296,92.417829,99.472296
2,1989,Vale Pump,0.000000,99.208443,93.568095,99.208443
3,1990,Vale Pump,0.000000,98.944591,92.292598,98.944591
4,1991,Vale Pump,0.000000,99.472296,92.417829,99.472296
5,1992,Vale Pump,0.000000,99.472296,92.417829,99.472296
6,1993,Vale Pump,0.000000,99.472296,92.417829,99.472296
7,1994,Vale Pump,0.000000,99.472296,92.417829,99.472296
8,1995,Vale Pump,0.000000,99.472296,92.417829,99.472296
9,1996,Vale Pump,0.000000,98.944591,88.547671,98.944591


,dates,DivName,class1_urban,class2_crops,contagion,largest_patch_index
0,1987,Wagner,13.043478,82.608696,60.808425,82.608696
1,1988,Wagner,13.043478,82.608696,60.808425,82.608696
2,1989,Wagner,13.043478,82.608696,60.808425,82.608696
3,1990,Wagner,13.043478,82.608696,60.808425,82.608696
4,1991,Wagner,13.043478,82.608696,60.808425,82.608696
5,1992,Wagner,13.043478,82.608696,60.808425,82.608696
6,1993,Wagner,13.043478,82.608696,60.808425,82.608696
7,1994,Wagner,13.043478,82.608696,60.808425,82.608696
8,1995,Wagner,21.739130,73.913043,53.790002,73.913043
9,1996,Wagner,21.739130,73.913043,53.790002,73.913043


,dates,DivName,class1_urban,class2_crops,contagion,largest_patch_index
0,1987,Warm Springs,14.597902,85.008741,71.278815,84.877622
1,1988,Warm Springs,15.909091,83.653846,70.032949,48.382867
2,1989,Warm Springs,16.302448,83.216783,69.564598,47.858392
3,1990,Warm Springs,17.001748,82.517483,68.677161,46.940559
4,1991,Warm Springs,18.225524,81.293706,67.927689,45.236014
5,1992,Warm Springs,19.361888,80.113636,66.823845,44.274476
6,1993,Warm Springs,19.536713,80.069930,67.324854,44.274476
7,1994,Warm Springs,19.580420,79.982517,67.049654,44.274476
8,1995,Warm Springs,19.842657,79.720280,66.785312,44.099650
9,1996,Warm Springs,19.798951,79.763986,66.856351,44.099650
